In [52]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as sco
plt.style.use('fivethirtyeight')
np.random.seed(777)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Get All NASDAQ Symbols

In [2]:
symbols = get_nasdaq_symbols()
all_symbols=symbols["Security Name"]
print(all_symbols.shape)
all_symbols.head()

(8910,)


Symbol
A                 Agilent Technologies, Inc. Common Stock
AA                        Alcoa Corporation Common Stock 
AAAU                         Perth Mint Physical Gold ETF
AACG    ATA Creativity Global - American Depositary Sh...
AADR                  AdvisorShares Dorsey Wright ADR ETF
Name: Security Name, dtype: object

# Read in data

In [3]:
data={}  # Save all raw data in this dict
missing=[] # Save a list of missing stock tickers
for stock_ticker in all_symbols.index[:20]:    # Only read in 20 entries for development purpose
    try:
        data[stock_ticker]=web.DataReader(stock_ticker, 'yahoo',start="2019-09-04")
        print(".", end ="") 
    except Exception as e:
        missing.append(stock_ticker)
        print(stock_ticker)
        continue

....................

In [4]:
len(data)

20

In [89]:
tickers=all_symbols.index[6:10]
weights=np.repeat(1/4,4)
tickers

Index(['AAMC', 'AAME', 'AAN', 'AAOI'], dtype='object', name='Symbol')

In [90]:
weights.sum()

1.0

# Equity Curve Function

In [91]:
def portfolio_equity(tickers,weights,backtest_window_s="2019-09-04",backtest_window_e="2019-11-06",benchmark="SPY"):
    if weights.sum()!=1:
        print("Incorrect Weight Vector!")
        return -202,-202,-202
    if len(weights)!=len(tickers):
        print("Unmatched Dimensions!")
        return -203,-203,-203
    data=pd.DataFrame()
    bench=web.DataReader(benchmark, 'yahoo',start=backtest_window_s,end=backtest_window_e)["Close"]
    bench=bench[1:]/bench[0]
    for stock_ticker in tickers:
        data[stock_ticker]=web.DataReader(stock_ticker, 'yahoo',start=backtest_window_s,end=backtest_window_e)["Close"]
        print(".", end ="") 
    data_pct=data.pct_change()[1:]
    daily_portfolio_return=np.dot(data_pct,weights)
    mean_return=data_pct.mean()
    covariance_matrix=data_pct.cov()
    
    equity_curve=np.cumprod(daily_portfolio_return+1)
    ec=pd.Series(equity_curve,index=bench.index)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ec.index,y=ec.values, name='Equity Curve',
                         line=dict(color='firebrick', width=4)))
    fig.add_trace(go.Scatter(x=bench.index,y=bench.values, name=benchmark,
                         line=dict(color='royalblue', width=4)))

# Edit the layout
    fig.update_layout(title='Portfolio Returns VS Benchmark',
                   xaxis_title='Date',
                   yaxis_title='Equity')

    return data,daily_portfolio_return,mean_return,covariance_matrix,fig

In [103]:
table,daily_portfolio_return,mean_return,covariance_matrix,fig=portfolio_equity(tickers,
                                                                   weights,
                                                                   backtest_window_s="2019-09-04",
                                                                   backtest_window_e="2019-11-06",
                                                                   benchmark="SPY")

....

In [104]:
fig.show()

# Efficient Prontier Portfolio Optimizers: Min_Var and Max_Sharpe 

In [118]:
returns=table.pct_change()
mean_returns=returns.mean()
cov_matrix=returns.cov()

In [119]:
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns
  
def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(4)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

In [120]:
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result

In [121]:
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result


In [122]:
def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result


def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

In [143]:
def display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, _ = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=table.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=table.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print("-"*80)
    print( "Maximum Sharpe Ratio Portfolio Allocation\n")
    print( "Annualised Return:", round(rp,2))
    print( "Annualised Volatility:", round(sdp,2))
    print( "\n")
    print( max_sharpe_allocation)
    print( "-"*80)
    print( "Minimum Volatility Portfolio Allocation\n")
    print( "Annualised Return:", round(rp_min,2))
    print( "Annualised Volatility:", round(sdp_min,2))
    print( "\n")
    print( min_vol_allocation)
     
    target = np.linspace(rp_min, 0.32, 50)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    
    
    fig=plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
    plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
    plt.title('Efficient Frontier of Portfolio')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)
    return fig,max_sharpe_allocation,min_vol_allocation,efficient_portfolios

In [ ]:
a,b,c,d=display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios=10000, risk_free_rate=0.02)

--------------------------------------------------------------------------------
Maximum Sharpe Ratio Portfolio Allocation

Annualised Return: 1.29
Annualised Volatility: 0.52


            AAMC  AAME  AAN  AAOI
allocation  48.4   0.0  0.0  51.6
--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Annualised Return: 0.07
Annualised Volatility: 0.29


             AAMC   AAME    AAN   AAOI
allocation  15.35  23.84  40.47  20.34
